# Script to calculate mean and std between the same configurations with different seeds

In [2]:
import pandas as pd

file_name = 'para_and_clause_results'

df = pd.read_csv('data/results/' + file_name + '.csv')

In [3]:
# Convert all bleu scores to percentages
for col in df.columns:
    if 'bleu' in col or 'f1' in col:
        df[col] = df[col] * 100

In [4]:
df['Name'] = df['Name'].str.replace('train-pred', 'train-a-pred')
# IF no-clause in Name then exp_num = 1, if calc-loss in Name then exp_num = 2, else 0
df['exp_num'] = df['Name'].apply(lambda x: 1 if 'no-clause' in x else 2 if 'calc-loss' in x else 0)

In [6]:


def prefix_columns(group):
    prefix_dict = {}
    for col in group.columns:
        if col[0] not in ['Name', 'Split']:
            prefix_dict[(f"{group.iloc[0]['Split'][0]}_{col[0]}", col[1])] = group[col].tolist()[0]
            prefix_dict[(f"{group.iloc[1]['Split'][0]}_{col[0]}", col[1])] = group[col].tolist()[1]
    return pd.Series(prefix_dict)

def combine_doc_random(_df):
    # drop seeds
    _df['Name'] = _df['Name'].str.split('_').str[:-3].str.join('_')

    # Average all values for the same Name (i.e. different seeds)
    _df = _df.groupby('Name').agg([('mean', pd.Series.mean), ('std', pd.Series.std)]).reset_index()

    _df['Split'] = _df['Name'].str.split('_').str[-2:-1].str.join('_')
    _df['Name'] = _df['Name'].str.split('_').str[:-2].str.join('_')


    # group by Name column and apply prefix_columns function
    combined_df = _df.groupby('Name').apply(prefix_columns).reset_index()

    return combined_df.sort_values(('doc_exp_num', 'mean'))


In [11]:
combined_df = combine_doc_random(df)

# round everything to one decimal place
combined_df = combined_df.round(1)


/var/folders/nh/sj47dvbx7p9dd_l626f7j3700000gn/T/ipykernel_78056/996426127.py:25: FutureWarning: ['State', 'Notes', 'User', 'Created', 'dataset', 'eval_strategy', 'evaluation_set', 'experiment', 'lr_scheduler_type', 'metric_for_best_model', 'metric_names', 'model_name', 'model_path', 'post_process', 'run_name', 'split', 'RANDOM_INPUT', 'RANDOM_INPUT_test', 'RANDOM_LABEL', 'RANDOM_LABEL_EVAL', 'RANDOM_LABEL_EVAL_test', 'RANDOM_LABEL_test', 'RANDOM_PRED', 'RANDOM_PRED_EVAL', 'RANDOM_PRED_EVAL_test', 'RANDOM_PRED_test', 'precisions', 'precisions_test', 'RANDOM_INPUT_test_oracle', 'RANDOM_LABEL_EVAL_test_oracle', 'RANDOM_LABEL_test_oracle', 'RANDOM_PRED_EVAL_test_oracle', 'RANDOM_PRED_test_oracle', 'precisions_test_oracle'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  _df = _df.groupby('Name').agg([('mean', pd.Series.mean), ('std', pd.Series.std)]).reset_index()


In [12]:
all_metrics = combined_df.loc[:, (((combined_df.columns.get_level_values(0).str.contains('test') & (combined_df.columns.get_level_values(0).str.contains('bleu') | combined_df.columns.get_level_values(0).str.contains('lrml') | combined_df.columns.get_level_values(0).str.contains('f1'))) | combined_df.columns.get_level_values(0).str.contains('Name') | combined_df.columns.get_level_values(0).str.contains('Runtime') | combined_df.columns.get_level_values(0).str.contains('lrml_f')))]
all_metrics

,Name,doc_lrml_f_score,random_lrml_f_score,doc_lrml_f_score,random_lrml_f_score,doc_lrml_f_score_test,random_lrml_f_score_test,doc_lrml_f_score_test,random_lrml_f_score_test,doc_bleu_test,...,doc_lrml_precision_test_oracle,random_lrml_precision_test_oracle,doc_lrml_recall_test_no_sep,random_lrml_recall_test_no_sep,doc_lrml_recall_test_no_sep,random_lrml_recall_test_no_sep,doc_lrml_recall_test_oracle,random_lrml_recall_test_oracle,doc_lrml_recall_test_oracle,random_lrml_recall_test_oracle
,,mean,mean,std,std,mean,mean,std,std,mean,...,std,std,mean,mean,std,std,mean,mean,std,std
0,t5-amr_0_para-and-clause,70.9,70.5,0.1,0.7,56.5,70.1,0.7,0.6,61.2,...,1.1,0.9,NaN,NaN,NaN,NaN,69.5,80.8,0.3,0.5


In [83]:
all_metrics.to_csv('data/results/' + file_name + '_combined.csv', index=False)